In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier

from PIL import Image

import itertools
import time

from BPGenerator import *

from matplotlib.pyplot import imshow
import numpy as np

from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.preprocessing.image import load_img, img_to_array
from keras import backend as K

Using TensorFlow backend.


In [2]:
# initialize the Bongard Problem generator to produce tiles of size 100x100
BPG = BPGenerator((100, 100))

In [3]:
# create training and testing data

BP_num = 61 # focus on BP#61

nb_train = 50000
nb_test = 10000
resizing = 96

X_train = []
y_train = []

for i in range(nb_train):
    side = "left" if i%2==0 else "right"

    # generate a random tile for this problem, resize it down to 20x20, and convert to vector
    x = BPG.flatten_img(BPG.draw_tile(BP_num, side).resize((resizing, resizing), Image.LANCZOS))
    label = 0 if side == "left" else 1

    X_train.append(x)
    y_train.append(label)

X_test = []
y_test = []

for i in range(nb_test):
    side = "left" if i%2==0 else "right"

    # generate a random tile for this problem, resize it down to 20x20, and convert to vector
    x = BPG.flatten_img(BPG.draw_tile(BP_num, side).resize((resizing, resizing), Image.LANCZOS))
    label = 0 if side == "left" else 1

    X_test.append(x)
    y_test.append(label)

In [4]:
x_train = np.asarray(X_train).astype('float32')
x_test = np.asarray(X_test).astype('float32')
x_train = np.reshape(X_train, (len(x_train), 96, 96, 1))  # adapt this if using `channels_first` image data format
x_test = np.reshape(X_test, (len(x_test), 96, 96, 1))  # adapt this if using `channels_first` image data format

In [5]:
# 搭建 autoencoder 模型

input_shape = (96, 96, 1)
model = Sequential()

# encoder 部分: ( conv + relu + maxpooling ) * 3
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape, padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Conv2D(8, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))

# decoder 部分: ( conv + relu + upsampling ) * 3 与 encoder 过程相反
model.add(Conv2D(8, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu',padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(1, kernel_size=(3, 3), activation='sigmoid', padding='same'))

model.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 96, 96, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 16)        4624      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 24, 24, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 8)         1160      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 12, 12, 8)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 12, 12, 8)         584       
__________

In [10]:
model.fit(x_train, x_train,
          epochs=20,
          batch_size=128,
          shuffle=True,
          validation_data=(x_test, x_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
40576/50000 [=======================>......] - ETA: 3:41 - loss: 0.0575 - acc: 0.9307

KeyboardInterrupt: 